# Imports

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.relative_locator import locate_with
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import os
import random
import requests

In [2]:
os.chdir('C:\Users\drkab\Documents\facemask')

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# driver = webdriver.Chrome("C:/Users/drkab/Documents/programs/chromedriver.exe")
url = 'https://www.google.com.sg/imghp?hl=en&tab=ri&authuser=0&ogbl'



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\drkab\.wdm\drivers\chromedriver\win32\101.0.4951.41]


In [4]:
driver.get(url)

In [59]:
search = driver.find_element(By.NAME,'q')
search.send_keys('friends tv series')
search.send_keys(Keys.RETURN)

In [25]:
content = driver.page_source
soup = BeautifulSoup(content,'html.parser')

In [26]:
def scroll_to_end(wd):
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

# Scrape images from Google Images

In [57]:
def scrape_images(max_images_to_download, driver): 
    count = 0
    start = 0
    img_urls = set()
    while(count<max_images_to_download):
        print('count = ', count)
        scroll_to_end(driver)
        img_thumbnails = driver.find_elements(By.XPATH, "//img[contains(@class,'Q4LuWd')]")
        tot_ths =len(img_thumbnails)
        print('total thumbnails found: ', tot_ths)
        
        count_img_scraped = 0
        for img in img_thumbnails[start:tot_ths]:
            try:
                img.click()
                time.sleep(2)
                actual_images = driver.find_elements(By.CSS_SELECTOR, 'img.n3VNCb')
                for actual_image in actual_images:
                    if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                        img_urls.add(actual_image.get_attribute('src'))
                        count_img_scraped += 1

                count = len(img_urls)

                if len(img_urls) >= max_images_to_download:
                    print(f"Finished scraping: {len(img_urls)} image links")
                    break
                time.sleep(random.randint(3, 10))
            except Exception as err:
    #             print(err)
                continue

        else:
            print("{} images scrapped in thumbnails from {} to {}".format(count_img_scraped, start, tot_ths))
            print("Scraped:", len(img_urls), "continuing ...")
            time.sleep(30)
            load_more_btn = driver.find_element(By.CSS_SELECTOR,".mye4qd")
            if load_more_btn:
                driver.execute_script("document.querySelector('.mye4qd').click();")
            end_text = driver.find_element(By.CSS_SELECTOR, ".DwpMZe ")
            if end_text.get_attribute('data-status') == "3":
                break

        start = len(img_thumbnails)
    return img_urls

In [60]:
# friends
img_urls = scrape_images(700, driver)

count =  0
total thumbnails found:  100
255 images scrapped in thumbnails from 0 to 100
Scraped: 172 continuing ...
count =  172
total thumbnails found:  312
582 images scrapped in thumbnails from 100 to 312
Scraped: 546 continuing ...
count =  546
total thumbnails found:  412
Finished scraping: 701 image links


In [61]:
print(len(img_urls))

701


# save images to local

In [73]:
def save_image(filepath,url):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        f = open(filepath, 'wb')
        f.write(image_content)
        f.close()
        print(f"SUCCESS - saved {url} - as {filepath}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [75]:
# img_urls = scrape_images(700, driver)
for i,url in enumerate(img_urls):
    save_image(os.path.join('data','friends',str(i)+'.jpg'), url)

SUCCESS - saved https://images2.minutemediacdn.com/image/upload/c_fill,w_720,ar_16:9,f_auto,q_auto,g_auto/shape/cover/sport/56565-gettyimages-908375-2de17b94795e304bb575f32f1d02fdd4.jpg - as data\friends\0.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSZ7lEvSYAlOupvTZyhAYsqPzIzmH78pofvuA&usqp=CAU - as data\friends\1.jpg
SUCCESS - saved https://rukminim2.flixcart.com/image/416/416/j7xngy80/poster/y/n/c/medium-official-friends-tv-series-umbrella-wall-decor-home-original-imaeyf5675wheufq.jpeg?q=70 - as data\friends\2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQT-VFdzOwvDB92dEODu_auyHuU_33GEzIBTw&usqp=CAU - as data\friends\3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTSayMnH0BYQcxE9zMsj05d6cv-2u_RVCsgAA&usqp=CAU - as data\friends\4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSHPdzcnbOwIL-WcG3bGONnzuslTZYNmzBkkw&usqp=CAU - as data\friends\5.jpg
SUCCESS - saved https://frie